# Exploring Hackers News Posts

In this project, I'll work with a data set of submissions to popular technology site [Hacker News](https://news.ycombinator.com/).

Hacker News is a site started by the startup incubator [Y Combinator](https://www.ycombinator.com/), where user-submitted stories (known as "posts") are voted and commented upon, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of Hacker News' listings can get hundreds of thousands of visitors as a result.

You can find the data set [here](https://www.kaggle.com/hacker-news/hacker-news-posts), but note that it has been reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions. Below are descriptions of the columns:

* `id`: The unique identifier from Hacker News for the post
* `title`: The title of the post
* `url`: The URL that the posts links to, if it the post has a URL
* `num_points`: The number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes
* `num_comments`: The number of comments that were made on the post
* `author`: The username of the person who submitted the post
* `created_at`: The date and time at which the post was submitted

I'm specifically interested in posts whose titles begin with either `Ask HN` or `Show HN`. Users submit `Ask HN` posts to ask the Hacker News community a specific question. Below are a couple examples:

```
Ask HN: How to improve my personal website?
Ask HN: Am I the only one outraged by Twitter shutting down share counts?
Ask HN: Aby recent changes to CSS that broke mobile?
```

Likewise, users submit `Show HN` posts to show the Hacker News community a project, product, or just generally something interesting. Below are a couple of examples:

```
Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform'
Show HN: Something pointless I made
Show HN: Shanhu.io, a programming playground powered by e8vm
```

I'll specifically compare these two types of posts to determine the following:

- Do `Ask HN` or `Show HN` receive more comments on average?
- Do posts created at a certain time receive more comments on average?

## Introduction

First, I'll read in the data and remove the headers.

In [21]:
# Import libraries
import datetime as dt
from csv import reader

with open("hacker_news.csv") as file:
  hn = list(reader(file))

# Remove the headers
headers = hn[0]
print(headers)

hn = hn[1:]

hn[:5]

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


[['12224879',
  'Interactive Dynamic Video',
  'http://www.interactivedynamicvideo.com/',
  '386',
  '52',
  'ne0phyte',
  '8/4/2016 11:52'],
 ['10975351',
  'How to Use Open Source and Shut the Fuck Up at the Same Time',
  'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/',
  '39',
  '10',
  'josep2',
  '1/26/2016 19:30'],
 ['11964716',
  "Florida DJs May Face Felony for April Fools' Water Joke",
  'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/',
  '2',
  '1',
  'vezycash',
  '6/23/2016 22:20'],
 ['11919867',
  'Technology ventures: From Idea to Enterprise',
  'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429',
  '3',
  '1',
  'hswarna',
  '6/17/2016 0:01'],
 ['10301696',
  'Note by Note: The Making of Steinway L1037 (2007)',
  'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0',
  '8',
  '2',
  'walterbell',
  '9/30/2015 4:12']]

We can see above that the data set contains the title of the posts, the number of comments for each post, and the date the post was created. Let's start by exploring the number of comments for each type of post. 

## Extracting Ask HN and Show HN Posts

First, I'll identify posts that begin with either `Ask HN` or `Show HN` and separate the data for those two types of posts into different lists. Separating the data makes it easier to analyze in the following steps.

In [22]:
# Identify posts that begin with either `Ask HN` or `Show HN` and separate the data into different lists
ask_posts, show_posts, other_posts = [], [], []

for row in hn:
  title = row[1].lower()
  if title.startswith("ask hn"):
    ask_posts.append(row)
  elif title.startswith("show hn"):
    show_posts.append(row)
  else:
    other_posts.append(row)

print("There are {} Ask HN posts".format(len(ask_posts)))
print("There are {} Show HN posts".format(len(show_posts)))
print("There are {} other posts".format(len(other_posts)))

There are 1744 Ask HN posts
There are 1162 Show HN posts
There are 17194 other posts


## Calculating the Average Number of Comments for Ask HN and Show HN Posts

Now that I separated ask posts and show posts into different lists, I'll calculate the average number of comments each type of post receives.

In [23]:
# Calculate the average number of comments for Ask HN posts
total_ask_comments = 0

for row in ask_posts:
  total_ask_comments += int(row[4])
avg_ask_comments = total_ask_comments/len(ask_posts)

print("The average number of comments for Ask HN posts is {0:.2f}".format(avg_ask_comments))

# Calculate the average number of comments for Show HN posts
total_show_comments = 0

for row in show_posts:
  total_show_comments += int(row[4])
avg_show_comments = total_show_comments/len(show_posts)

print("The average number of comments for Show HN posts is {0:.2f}".format(avg_show_comments))

The average number of comments for Ask HN posts is 14.04
The average number of comments for Show HN posts is 10.32


On average, ask posts in our sample receive approximately 14 comments, whereas show posts receive approximately 10. Since ask posts are more likely to receive comments, we'll focus our remaining analysis just on these posts. 

## Finding the Amount of Ask Posts and Comments by Hour Created

Next, I'll determine if we can maximize the amount of comments an ask post receives by creating it at a certain time. First, I'll find the amount of ask posts created during each hour of day, along with the number of comments those posts received. Then, I'll calculate the average amount of comments ask posts created at each hour of the day receive.

In [24]:
# Calculate the amount of ask posts created during each hour of day and the number of comments received
result_list = []

for row in ask_posts:
  result_list.append([row[6], int(row[4])])

counts_by_hour, comments_by_hour = {}, {}

for row in result_list:
  hour = dt.datetime.strptime(row[0], "%m/%d/%Y %H:%M")
  hour = hour.strftime("%H")

  if not hour in counts_by_hour:
    counts_by_hour[hour] = 1
    comments_by_hour[hour] = row[1]
  else:
    counts_by_hour[hour] += 1
    comments_by_hour[hour] += row[1]

comments_by_hour

{'00': 447,
 '01': 683,
 '02': 1381,
 '03': 421,
 '04': 337,
 '05': 464,
 '06': 397,
 '07': 267,
 '08': 492,
 '09': 251,
 '10': 793,
 '11': 641,
 '12': 687,
 '13': 1253,
 '14': 1416,
 '15': 4477,
 '16': 1814,
 '17': 1146,
 '18': 1439,
 '19': 1188,
 '20': 1722,
 '21': 1745,
 '22': 479,
 '23': 543}

## Calculating the Average Number of Comments for Ask HN Posts by Hour

In [25]:
# Calculate the average amount of comments `Ask HN` posts created at each hour of the day receive
avg_by_hour = []

for key, val in comments_by_hour.items():
  avg_by_hour.append([key, val/counts_by_hour[key]])

avg_by_hour

[['09', 5.5777777777777775],
 ['13', 14.741176470588234],
 ['10', 13.440677966101696],
 ['14', 13.233644859813085],
 ['16', 16.796296296296298],
 ['23', 7.985294117647059],
 ['12', 9.41095890410959],
 ['17', 11.46],
 ['15', 38.5948275862069],
 ['21', 16.009174311926607],
 ['20', 21.525],
 ['02', 23.810344827586206],
 ['18', 13.20183486238532],
 ['03', 7.796296296296297],
 ['05', 10.08695652173913],
 ['19', 10.8],
 ['01', 11.383333333333333],
 ['22', 6.746478873239437],
 ['08', 10.25],
 ['04', 7.170212765957447],
 ['00', 8.127272727272727],
 ['06', 9.022727272727273],
 ['07', 7.852941176470588],
 ['11', 11.051724137931034]]

## Sorting and Printing Values from a List of Lists

In [26]:
# Sort the values and print the the 5 hours with the highest average comments
swap_avg_by_hour = []

for row in avg_by_hour:
  swap_avg_by_hour.append(row[::-1])

sorted_swap = sorted(swap_avg_by_hour, reverse=True)

print("Top 5 Hours for Ask Posts Comments\n")

for row in sorted_swap[:5]:
  print("{} - {:.2f} average number of comments per post".format(dt.datetime.strptime(row[1], "%H").strftime("%H:%M"), row[0]))

Top 5 Hours for Ask Posts Comments

15:00 - 38.59 average number of comments per post
02:00 - 23.81 average number of comments per post
20:00 - 21.52 average number of comments per post
16:00 - 16.80 average number of comments per post
21:00 - 16.01 average number of comments per post


The hour that receives the most comments per post on average is 15:00, with an average of 38.59 comments per post. There's about a 60% increase in the number of comments between the hours with the highest and second highest 
average number of comments.

According to the data set [documentation](https://www.kaggle.com/hacker-news/hacker-news-posts/home), the timezone used is Eastern Time in the US. So, we could also write 15:00 as 3:00 pm est.

# Conclusion

In this project, I analyzed ask posts and show posts to determine which type of post and time receive the most comments on average. Based on the analysis, to maximize the amount of comments a post receives, I would recommend the post be categorized as ask post and created between 15:00 and 16:00 (3:00 pm est - 4:00 pm est). 

However, it should be noted that the data set I analyzed excluded posts without any comments. Given that, it's more accurate to say that *of the posts that received comments*, ask posts received more comments on average and ask posts created between 15:00 and 16:00 (3:00 pm est - 4:00 pm est) received the most comments on average.